In [ ]:
data = [
    ['intel', 3, 'Apple'],
    ['amd', 3, 'Apple'],
    ['mediatek', 1, 'hp'],
    ['mediatek', 1, 'hp'],
    ['amd', 3, 'Lenovo'],
]

In [ ]:
header = ["processor", "power", "label"]

In [ ]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [ ]:
unique_vals(data, 0)

{'amd', 'intel', 'mediatek'}

In [ ]:
def class_counts(rows):
    """    Counbts   the number of each type of example in a dataset.  """
    counts = {} 
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [ ]:
class_counts(data)

{'Apple': 2, 'Lenovo': 1, 'hp': 2}

In [ ]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [ ]:
class Question:
    """ to partition a dataset.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))


In [ ]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [ ]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [ ]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)


In [ ]:
def find_best_split(rows):
    best_gain = 0  
    best_question = None 
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 

    for col in range(n_features): 

        values = set([row[col] for row in rows]) 

        for val in values: 

            question = Question(col, val)

            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [ ]:
class Leaf:
    """A Leaf node classifies data.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
class Decision_Node:
    """A Decision Node asks a question.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
def build_tree(rows):
    """Builds the tree.
    """

    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [ ]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return


    print (spacing + str(node.question))
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [ ]:
my_tree = build_tree(data)

In [ ]:
print_tree(my_tree)

Is power >= 3?
--> True:
  Is processor == amd?
  --> True:
    Predict {'Apple': 1, 'Lenovo': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'hp': 2}


In [ ]:
def classify(row, node):

    if isinstance(node, Leaf):
        return node.predictions


    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [ ]:
def print_leaf(counts):
    """to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [ ]:
# this is new data diferent from the one which was being used as letting the algo predict the original
data = [
    ['intel', 3, 'Apple'],
    ['amd', 4, 'Apple'],
    ['mediatek', 2, 'hp'],
    ['mediatek', 1, 'hp'],
    ['amd', 3, 'Lenovo'],
]

In [ ]:
for row in data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lenovo': '50%'}
Actual: hp. Predicted: {'hp': '100%'}
Actual: hp. Predicted: {'hp': '100%'}
Actual: Lenovo. Predicted: {'Apple': '50%', 'Lenovo': '50%'}
